# Como lake data exploration

Data obtained from the archive of *ARPA Lombardia* ([link](https://www.arpalombardia.it/Pages/Ricerca-Dati-ed-Indicatori.aspx)), we will use tables "Afflussi settimanali per bacino" for all the available years (2011-2018), expressed in $m^3 \cdot 10^6$.

In [1]:
import pandas as pd
import os
import datetime

First, import the different datasets and build the complete one:

In [258]:
# Get the names of the files
filenames = os.listdir('data')
filenames.sort()

# Get data from the files and concatenate them in a single df
full_df = pd.DataFrame()

for name in list(filenames):
    
    # Read data
    print(name)
    df = pd.read_excel('data/'+name, 
                       skiprows = range(0,12),
                       usecols = 'A,I,J')
    
    # Remove empty rows
    df = df.dropna(axis=0, how='all') 
    
    # Split week interval into two columns (start and end week)
    # (We have to be careful because date written in 2 different ways in the files)
    if df['SETTIMANA'][0][:5] == "01/01":
        df['From'] = df['SETTIMANA'].str[:5]
        df['To']   = df['SETTIMANA'].str[-5:]
    elif df['SETTIMANA'][0][:3] == "1/1":
        df[['From','To']] = df['SETTIMANA'].str.split("-",expand=True,)
    else:
        print('\n\n There is a problem in the date format \n\n')
        
    df.drop(['SETTIMANA'], axis=1)
    df = df[['From','To','ADDA PRELACUALE','MERA']]
    
    # Convert dates strings into datetimes
    year = name[14:18]
    df['From'] = df['From'] + '/' + year
    df['To']   = df['To']   + '/' + year
    df['From'] = pd.to_datetime(df['From'], format='%d/%m/%Y')
    df['To']   = pd.to_datetime(df['To'], format='%d/%m/%Y')
    
    # Build full dataset to contain all years
    full_df = pd.concat([full_df,df]).reset_index(drop=True)
    
    
# Reset indexes of the full dataset 
# (before, indexes restart at each year) 
full_df = full_df.reset_index(drop=True)
#orig_df = full_df

afflussi_sett_2011.xlsx
afflussi_sett_2012.xlsx
afflussi_sett_2013.xlsx
afflussi_sett_2014.xlsx
afflussi_sett_2015.xlsx
afflussi_sett_2016.xlsx
afflussi_sett_2017.xlsx
afflussi_sett_2018.xlsx


At the moment each row represents the weekly inflows but, if a week is spanned between two moths/years, it is splitted in two entries.</br>
Now we will join those portions of weeks to have a uniform unit of time (a week).

In [259]:
# Find rows that do not represent full weeks (pairs of rows)
week_len = (full_df['To'] - full_df['From'])[1:-1]
partial_weeks = week_len.index[week_len!=datetime.timedelta(days=6)][::2]

# Add inflow values between these splitted weeks
inflows = (full_df.reset_index()
                  .replace({'index': dict(zip(partial_weeks+1,partial_weeks))})
                  .groupby('index', sort=False).sum() )

# Update end-date of the first portion of these weeks
full_df['To'][partial_weeks] = full_df['To'][partial_weeks+1]

# Remove second portions of the partial weeks and add complete inflows
full_df[['ADDA PRELACUALE','MERA']] = inflows
full_df = full_df.drop(partial_weeks+1)


<ipython-input-259-7a65947efdb5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['To'][partial_weeks] = full_df['To'][partial_weeks+1]


In [260]:
### TEST ###

# Find rows that do not represent full weeks (pairs of rows)
week_len = (full_df['To'] - full_df['From'])[1:-1]
partial_weeks = week_len.index[week_len!=datetime.timedelta(days=6)][::2]
partial_weeks

Int64Index([178], dtype='int64')

In [261]:
full_df

,From,To,ADDA PRELACUALE,MERA
0,2011-01-01,2011-01-02,0.0,0.1
1,2011-01-03,2011-01-09,71.4,41.1
2,2011-01-10,2011-01-16,53.7,13.2
3,2011-01-17,2011-01-23,0.2,0.2
4,2011-01-24,2011-01-30,0.3,0.1
...,...,...,...,...
487,2018-12-03,2018-12-09,15.4,7.6
488,2018-12-10,2018-12-16,3.1,0.9
489,2018-12-17,2018-12-23,13.1,6.4
490,2018-12-24,2018-12-30,8.6,4.7
